In [1]:
# !pip install --quiet --pre  dgl-cu101
# !pip install --quiet torch==1.6.0

In [2]:
import argparse
import itertools
import os
import numpy as np
from numpy import save,load,savetxt,loadtxt,savez_compressed
from sklearn.metrics import roc_auc_score, f1_score,average_precision_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc as auc_score
import pandas as pd
import scipy.sparse as sp
import time
from tqdm import tqdm, tqdm_notebook,tnrange
tqdm.pandas(position=0, leave=True)
import math 

import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import dgl.function as fn
from dgl.ops import edge_softmax

from functools import partial
import seaborn as sns
import pickle
import random
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
import warnings
warnings.filterwarnings('ignore')
import utils
import tsne_func
print("torch version is {}".format(th.__version__))
print("DGL version is {}".format(dgl.__version__))

Using backend: pytorch


torch version is 1.6.0
DGL version is 0.6a210127


In [3]:
def seed_everything(seed):
    random.seed(seed)
    th.manual_seed(seed)
    th.cuda.manual_seed_all(seed)
    th.backends.cudnn.deterministic = True
    th.backends.cudnn.benchmark = False
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(101)

In [4]:
KG_dir="/workspace/cjiang/eagle_project/CAP_graph/dataset/"
start=time.time()
with open(os.path.join(KG_dir,'CAP_Graph'), 'rb') as f:
    G, node_labels = pickle.load(f)
end=time.time()
print("It took {:0.4f} seconds to load graph".format(end-start))

It took 13.5197 seconds to load graph


In [5]:
utils.graph_show(G)

**************************************************
Node_types:  ['usaanr', 'zipcode']
Edge_types:  ['AUTO_RELATED', 'Brother_Sister', 'Busi_rel_Other', 'Child', 'Ex-Spouse', 'Located_In', 'Parent', 'Pers_rel_Other', 'SPONSEE', 'SPONSOR', 'Spouse', 'Step-Child', 'Step-Parent', 'Location_of']
**************************************************
Canonical Etypes of Graph is:

usaanr              AUTO_RELATED        usaanr              
usaanr              Brother_Sister      usaanr              
usaanr              Busi_rel_Other      usaanr              
usaanr              Child               usaanr              
usaanr              Ex-Spouse           usaanr              
usaanr              Located_In          zipcode             
usaanr              Parent              usaanr              
usaanr              Pers_rel_Other      usaanr              
usaanr              SPONSEE             usaanr              
usaanr              SPONSOR             usaanr              
usaanr          

In [6]:
usaanr_feat=[]
for key, scheme in G.node_attr_schemes(ntype="usaanr").items():
    usaanr_feat.append(key)
usaanr_feat=[x for x in usaanr_feat if x not in ['type','usaanr','train_mask','val_mask','test_mask']]

print()
print("The features associated with USAA Member are\n ")
for i in usaanr_feat:
    print(i)


The features associated with USAA Member are
 
usaayr
AGE_BAND
ORIGEL
ELIG2
cmpyelig
SEX
MARST
BRANCH
ENLPAYGD
MILST
MLIST_OrigStat
ACTCORP
STATE
Segment


In [7]:
## USAA Members Features Embedding
class USAANR_Embedding(nn.Module):
    def __init__(self,G,feature_size):
        super(USAANR_Embedding,self).__init__()
        self.G=G
        self.feature_size=feature_size
        ## Embedding matrices for features of nodes.
        self.emb = nn.ModuleDict()
        
        
        for i,col in enumerate(usaanr_feat):
            self.emb[col]=nn.Embedding(G.nodes['usaanr'].data[col].max().item()+1, feature_size)
        ## Embedding for the node of House Properties
#         self.node_emb=nn.Embedding(G.filter_nodes(lambda nodes: (nodes.data['node_type'] == 0).squeeze(1),\
#                                        ntype='House_Properties').max().item() + 1, feature_size)
        self.node_emb=nn.Embedding(2, feature_size)
    
    def forward(self,nid):
        nid=nid.to("cpu")
        h=self.node_emb(self.G.nodes['usaanr'].data['type'][nid].squeeze(1).to(device))
        extra_repr=[]
        for i,col in enumerate(usaanr_feat):
            ndata=self.G.nodes['usaanr'].data[col]
            extra_repr.append(self.emb[col](ndata[nid].to(device)).squeeze(1))
        return h + th.stack(extra_repr, 0).sum(0)

## zipcode Embedding
class Zipcode_Embedding(nn.Module):
    def __init__(self,G,feature_size):
        super(Zipcode_Embedding,self).__init__()
        self.G=G
        self.feature_size=feature_size
        
        ## Embedding for the node of zipcode.
#         self.node_emb=nn.Embedding(G.filter_nodes(lambda nodes: (nodes.data['type'] == 1).squeeze(1),\
#                                        ntype='zipcode').max().item() + 1, feature_size)
        self.node_emb=nn.Embedding(2, feature_size)
    
    def forward(self,nid):
        nid=nid.to("cpu")
        h=self.node_emb(self.G.nodes['zipcode'].data['type'][nid].squeeze(1).to(device))
        return h 

class HGTLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 node_dict,
                 edge_dict,
                 n_heads,
                 dropout = 0.2,
                 use_norm = False):
        super(HGTLayer, self).__init__()

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.node_dict     = node_dict
        self.edge_dict     = edge_dict
        self.num_types     = len(node_dict)
        self.num_relations = len(edge_dict)
        self.total_rel     = self.num_types * self.num_relations * self.num_types
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None

        self.k_linears   = nn.ModuleList()
        self.q_linears   = nn.ModuleList()
        self.v_linears   = nn.ModuleList()
        self.a_linears   = nn.ModuleList()
        self.norms       = nn.ModuleList()
        self.use_norm    = use_norm

        for t in range(self.num_types):
            self.k_linears.append(nn.Linear(in_dim,   out_dim))
            self.q_linears.append(nn.Linear(in_dim,   out_dim))
            self.v_linears.append(nn.Linear(in_dim,   out_dim))
            self.a_linears.append(nn.Linear(out_dim,  out_dim))
            if use_norm:
                self.norms.append(nn.LayerNorm(out_dim))

        self.relation_pri   = nn.Parameter(th.ones(self.num_relations, self.n_heads))
        self.relation_att   = nn.Parameter(th.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg   = nn.Parameter(th.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.skip           = nn.Parameter(th.ones(self.num_types))
        self.drop           = nn.Dropout(dropout)

        nn.init.xavier_uniform_(self.relation_att)
        nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, G, h):
        with G.local_scope():
            node_dict, edge_dict = self.node_dict, self.edge_dict
            for srctype, etype, dsttype in G.canonical_etypes:
                sub_graph = G[srctype, etype, dsttype]

                k_linear = self.k_linears[node_dict[srctype]]
                v_linear = self.v_linears[node_dict[srctype]]
                q_linear = self.q_linears[node_dict[dsttype]]

                k = k_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                v = v_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                q = q_linear(h[dsttype]).view(-1, self.n_heads, self.d_k)

                e_id = self.edge_dict[etype]

                relation_att = self.relation_att[e_id]
                relation_pri = self.relation_pri[e_id]
                relation_msg = self.relation_msg[e_id]

                k = th.einsum("bij,ijk->bik", k, relation_att)
                v = th.einsum("bij,ijk->bik", v, relation_msg)

                sub_graph.srcdata['k'] = k
                sub_graph.dstdata['q'] = q
                sub_graph.srcdata['v'] = v

                sub_graph.apply_edges(fn.v_dot_u('q', 'k', 't'))
                attn_score = sub_graph.edata.pop('t').sum(-1) * relation_pri / self.sqrt_dk
                attn_score = edge_softmax(sub_graph, attn_score, norm_by='dst')

                sub_graph.edata['t'] = attn_score.unsqueeze(-1)

            G.multi_update_all({etype : (fn.u_mul_e('v', 't', 'm'), fn.sum('m', 't')) \
                                for etype in edge_dict}, cross_reducer = 'mean')

            new_h = {}
            for ntype in G.ntypes:
                '''
                    Step 3: Target-specific Aggregation
                    x = norm( W[node_type] * gelu( Agg(x) ) + x )
                '''
                n_id = node_dict[ntype]
                alpha = th.sigmoid(self.skip[n_id])
                t = G.nodes[ntype].data['t'].view(-1, self.out_dim)
                trans_out = self.drop(self.a_linears[n_id](t))
                trans_out = trans_out * alpha + h[ntype] * (1-alpha)
                if self.use_norm:
                    new_h[ntype] = self.norms[n_id](trans_out)
                else:
                    new_h[ntype] = trans_out
            return new_h
        
class HGT(nn.Module):
    def __init__(self, G, node_dict, edge_dict, in_feat, h_dim, out_feat, n_layers, n_heads, use_norm = True):
        super(HGT, self).__init__()
        self.G=G
        self.node_dict = node_dict
        self.edge_dict = edge_dict
        self.gcs = nn.ModuleList()
        self.in_feat = in_feat
        self.h_dim = h_dim
        self.out_feat = out_feat
        self.n_layers = n_layers
        self.adapt_ws  = nn.ModuleList()
        for t in range(len(node_dict)):
            self.adapt_ws.append(nn.Linear(in_feat,   h_dim))
        for _ in range(n_layers):
            self.gcs.append(HGTLayer(h_dim, h_dim, node_dict, edge_dict, n_heads, use_norm = use_norm))
        self.out = nn.Linear(h_dim, out_feat)
        
        self.node_embed=nn.ModuleDict()
        self.node_embed['usaanr'] = USAANR_Embedding(self.G,self.in_feat)
        self.node_embed['zipcode'] = Zipcode_Embedding(self.G,self.in_feat)
           
    def forward(self, G, out_key):
        H = {}
        for ntype in G.ntypes:
            nid = th.arange(G.num_nodes(ntype))
            H[ntype] = F.gelu(self.adapt_ws[self.node_dict[ntype]](self.node_embed[ntype](nid)))
        
        for layer in self.gcs:
            H = layer(self.G, H)

        return self.out(H[out_key]), H[out_key]      


In [8]:
#### Create subgraph for the purpose of preliminary test
# dict_nodes={"usaanr":th.arange(G.num_nodes('usaanr'))[0:1000],'zipcode':th.arange(G.num_nodes('zipcode'))[0:100]}
# sg=dgl.node_subgraph(G,dict_nodes)

# dict_edges={}
# for etype in G.etypes:
#     dict_edges[etype]=th.arange(G.num_edges(etype))[0:5000]
# sg=dgl.edge_subgraph(G,dict_edges)

In [9]:
# utils.graph_show(sg)

In [10]:
# sg.nodes['usaanr'].data["_ID"].numpy().shape,  node_labels.shape, node_labels[sg.nodes['usaanr'].data["_ID"]].shape

#### settings

In [11]:
parser = argparse.ArgumentParser(description='HGT')
parser.add_argument("--dropout", type=float, default=0,
        help="dropout probability")
parser.add_argument("--h_dim", type=int, default=128,
        help="number of hidden units")
parser.add_argument("--out_dim", type=int, default=1,
        help="output dimension")
parser.add_argument("--gpu", type=int, default=0,
        help="gpu")
parser.add_argument("--lr", type=float, default=1e-5,
        help="learning rate")
parser.add_argument('--clip',    type=int, default=1.0) 
parser.add_argument('--max_lr',  type=float, default=1e-3) 

parser.add_argument("--num_layers", type=int, default=1,
        help="number of propagation rounds")
parser.add_argument("-e", "--n_epochs", type=int, default=1,
        help="number of training epochs")
parser.add_argument("--model_path", type=str, default="/workspace/cjiang/eagle_project/CAP_graph/hgt_model_param.pt",
        help='path for save the model')
parser.add_argument("--l2norm", type=float, default=0,
        help="l2 norm coef")

parser.add_argument("--batch-size", type=int, default=1024,
        help="Mini-batch size. If -1, use full graph training.")
parser.add_argument("--num_mini_batch", type=int, default=8,
        help="Number of minibatch.")
parser.add_argument("--fanout", type=int, default=None,
        help="Fan-out of neighbor sampling.")

parser.add_argument("--seed",  type=int,default=101,
        help="random seed for np.random.seed, torch.manual_seed and torch.cuda.manual_seed.")

parser.add_argument("--num_worker",  type=int,default=4,  
        help="number of worker for neighbor sampling") 

args,unknown=parser.parse_known_args()

args.num_layers=1
args.dropout=0.2
args.lr=1e-3
args.l2norm=1e-3
args.n_epochs=20
args.h_dim=64
args.batch_size=1024
print(args)

Namespace(batch_size=1024, clip=1.0, dropout=0.2, fanout=None, gpu=0, h_dim=64, l2norm=0.001, lr=0.001, max_lr=0.001, model_path='/workspace/cjiang/eagle_project/CAP_graph/hgt_model_param.pt', n_epochs=20, num_layers=1, num_mini_batch=8, num_worker=4, out_dim=1, seed=101)


#### setting up training, validation and test set

In [12]:
num_rels=len(G.etypes)
# LABEL=th.tensor(node_labels[sg.nodes['usaanr'].data["_ID"]]).long()
LABEL=th.tensor(node_labels).long()
labels, count=th.unique(LABEL,return_counts=True)
num_classes=labels.shape[0]
pd.DataFrame({"label_class":labels, "count":count}).style.format({'count':'{:,}'})

,label_class,count
0,0,"20,500,666"
1,1,"262,009"
2,2,"226,935"
3,3,"2,767"
4,4,"2,435,787"
5,5,"1,461,264"
6,6,"2,938,245"
7,7,"56,659"


In [13]:
train_mask=G.nodes["usaanr"].data.pop('train_mask')
val_mask=G.nodes["usaanr"].data.pop('val_mask')
test_mask=G.nodes["usaanr"].data.pop('test_mask')

train_idx=th.nonzero(train_mask.squeeze(1)).numpy()
val_idx=th.nonzero(val_mask.squeeze(1)).numpy()
test_idx=th.nonzero(test_mask.squeeze(1)).numpy()

train_idx=th.from_numpy(train_idx).squeeze(1)    
val_idx=th.from_numpy(val_idx).squeeze(1)    
test_idx=th.from_numpy(test_idx).squeeze(1)

train_label=LABEL[train_idx]
val_label=LABEL[val_idx]
test_label=LABEL[test_idx]

print('{:<15} {:<10,}'.format("Training set",train_idx.shape[0]))
print('{:<15} {:<10,}'.format("validation set",val_idx.shape[0]))
print('{:<15} {:<10,}'.format("test set",test_idx.shape[0]))

Training set    22,307,469
validation set  2,788,435 
test set        2,788,428 


In [14]:
assert train_idx.shape[0]+val_idx.shape[0]+test_idx.shape[0] == G.num_nodes('usaanr')

In [15]:
##### check cuda
device="cpu"
# use_cuda=args.gpu>=0 and th.cuda.is_available()
# if use_cuda:
#     th.cuda.set_device(args.gpu)
#     device='cuda:%d' % args.gpu
print(device)

cpu


In [16]:
node_dict = {}
edge_dict = {}
for ntype in G.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in G.etypes:
    edge_dict[etype] = len(edge_dict)
    G.edges[etype].data['id'] = th.ones(G.number_of_edges(etype), dtype=th.long) * edge_dict[etype] 

print(node_dict)
print(edge_dict)

{'usaanr': 0, 'zipcode': 1}
{'AUTO_RELATED': 0, 'Brother_Sister': 1, 'Busi_rel_Other': 2, 'Child': 3, 'Ex-Spouse': 4, 'Located_In': 5, 'Parent': 6, 'Pers_rel_Other': 7, 'SPONSEE': 8, 'SPONSOR': 9, 'Spouse': 10, 'Step-Child': 11, 'Step-Parent': 12, 'Location_of': 13}


In [17]:
for etype in G.etypes:
    for key,val in G.edges[etype].data.items():
        if key=="id":
            print("{:<15}{:<10}{:<10}".format(etype,key,th.unique(val).item()))

AUTO_RELATED   id        0         
Brother_Sister id        1         
Busi_rel_Other id        2         
Child          id        3         
Ex-Spouse      id        4         
Located_In     id        5         
Parent         id        6         
Pers_rel_Other id        7         
SPONSEE        id        8         
SPONSOR        id        9         
Spouse         id        10        
Step-Child     id        11        
Step-Parent    id        12        
Location_of    id        13        


In [18]:
# create model

model = HGT(G,
            node_dict, 
            edge_dict,
            in_feat=args.h_dim,
            h_dim=args.h_dim,
            out_feat=num_classes,
            n_layers=2,
            n_heads=4,
            use_norm = True)

# if use_cuda:
#     model.cuda()

model.gcs

ModuleList(
  (0): HGTLayer(
    (k_linears): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
    (q_linears): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
    (v_linears): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
    (a_linears): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_features=64, out_features=64, bias=True)
    )
    (norms): ModuleList(
      (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (drop): Dropout(p=0.2, inplace=False)
  )
  (1): HGTLayer(
    (k_linears): ModuleList(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): Linear(in_f

In [19]:
# optimizer = th.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2norm)
optimizer = th.optim.AdamW(model.parameters())
scheduler = th.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.n_epochs, max_lr = args.max_lr)

In [20]:
print("The total # of parameter is {:,}".format(sum([p.nelement() for p in model.parameters()]) ) )

The total # of parameter is 151,740


In [21]:
param_dict={n: p.nelement() for n, p in model.named_parameters()}
for i,j in param_dict.items():
    print("{:<70}{:<15,}".format(i,j))

gcs.0.relation_pri                                                    56             
gcs.0.relation_att                                                    14,336         
gcs.0.relation_msg                                                    14,336         
gcs.0.skip                                                            2              
gcs.0.k_linears.0.weight                                              4,096          
gcs.0.k_linears.0.bias                                                64             
gcs.0.k_linears.1.weight                                              4,096          
gcs.0.k_linears.1.bias                                                64             
gcs.0.q_linears.0.weight                                              4,096          
gcs.0.q_linears.0.bias                                                64             
gcs.0.q_linears.1.weight                                              4,096          
gcs.0.q_linears.1.bias                                

In [22]:
# %pdb
best_val_acc = 0
best_test_acc = 0
train_step = 0

# th.manual_seed(args.seed)
# th.cuda.manual_seed(args.seed)
# th.cuda.manual_seed_all(args.seed)
# np.ranom.seed(args.seed)
# random.seed(args.seed)
# th.backends.cudnn.deterministic=True

# for epoch in tqdm(range(0,args.n_epochs)):
for epoch in range(0,args.n_epochs):
    
    model.train()
    H=[]
    
    #====================================#
    #            Traning                 #
    #====================================#

    t0 = time.time()

    logits,h = model(G,'usaanr')
    optimizer.zero_grad()
#         loss = F.cross_entropy(logits, labels_train.squeeze(1),weight=th.Tensor([1,args.weight]).to(device))
    loss = F.cross_entropy(logits[train_idx], LABEL[train_idx].squeeze().to(device))

    loss.backward()
    th.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
    optimizer.step()

    train_step += 1
    scheduler.step(train_step)
    
    if epoch % 1 == 0:
        model.eval()
        logits,h = model(G,'usaanr')
        pred   = logits.argmax(1).cpu()
        train_acc = (pred[train_idx] == LABEL[train_idx]).float().mean()
        val_acc   = (pred[val_idx]   == LABEL[val_idx]).float().mean()
        test_acc  = (pred[test_idx]  == LABEL[test_idx]).float().mean()
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        t1 = time.time()
        print('Epoch: %d | LR: %.5f | Loss %.4f | Train Acc %.4f | Val Acc %.4f | (Best %.4f) | Test Acc %.4f | (Best %.4f) | Elapsed: %.4f' % (
            epoch,
            optimizer.param_groups[0]['lr'], 
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
            t1-t0
        ))


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 92799056896 bytes. Error code 12 (Cannot allocate memory)

In [1]:
24561910*64/

1571962240

In [2]:
24 * 2**20 * 64 * 4 / 2**30

6.0

In [5]:
2**30

1073741824